In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import uproot
import matplotlib as mpl
import math

from scipy.optimize import curve_fit
from scipy import interpolate, special
from scipy.stats import norm as normal

import cProfile

In [ ]:
# MPL config
mpl.rc('font', size=14)

dosave = False
savedir = "/home/london/icarus/field_response/figs/"

# Clear figures in main loop
CLEARPLT = False

In [ ]:
# Configuration

# Plane
IPLANE = 2

# Parameters for measurement resolution
if IPLANE == 2:
    SIGMA_A = 0.31
    SIGMA_B = 0.43
elif IPLANE == 1:
    SIGMA_A = 0.67
    SIGMA_B = 0.59
elif IPLANE == 0:
    SIGMA_A = 0.74
    SIGMA_B = 0.65

# Central index of electron path response
if IPLANE == 2:
    CENTER_IND = 625 # not banded
    # CENTER_IND = 610
elif IPLANE == 1:
    # CENTER_IND = 608 # not banded
    CENTER_IND = 595 # banded
elif IPLANE == 0:
    CENTER_IND = 595

# Filename for WC configuration containing electron paths
WCpaths_filename_nom = "./garfield-icarus-fnal-rev1.json"
#Sergey's WC paths
# WCpaths_filename_nom = "./icarus_fnal_band.json"

WCpaths_filename_noY = "./icarus_fnal_noY.json"

# Time spacing in file
time = np.linspace(0,100,1000, endpoint=False)

# Filename and histogram with electronics response
fER = "elecResp_ICARUS.root"
fERh = uproot.open(fER)["ER"]

# Filename with simulated signal response
SR_filename = "./mc/WFresults_Plane%i_WC.root" % IPLANE

angle_min = 20
angle_max = 80
angle_increment = 2
all_angles = np.array(list(range(angle_min, angle_max, angle_increment))) # 20-80, inclusive

# Whether to include Guassian broadening in fit
FITGAUS = False

# Include skew?
FITSKEW = False

# What range of time to fit the response
FIT_PERIOD = 5

# What range of time to plot the response
PLT_PERIOD = 12

# Plot various things
PLTPATHS = False # electron paths
PLTFR = False # field response
PLTSR = False # signal response
PLTFIT = False # fit

#Time increment
dt = time[1] - time[0]

In [ ]:
# FIT_ANGLES = np.array([30, 36, 40, 46, 50, 54, 60, 64])
FIT_ANGLES = np.array(all_angles[3:-7])
FIT_ANGLES

In [ ]:
# Helper functions
def convolve(f1, f2):
    '''
    Return the simple convolution of the two arrays using FFT+mult+invFFT method.
    '''
    # fftconvolve adds an unwanted time shift
    #from scipy.signal import fftconvolve
    #return fftconvolve(field, elect, "same")
    s1 = np.fft.fft(f1)
    s2 = np.fft.fft(f2)
    sig = np.fft.ifft(s1*s2)

    return np.real(sig)

def gaus(t, sigma):
    return np.exp(-t**2/(2*sigma**2))

def skewgaus(t, sigma, skew):
    skew = -1/skew
    return gaus(t, sigma)*(1 + special.erf(skew*t/np.sqrt(2)))

def agaus(t, sigmalo, sigmahi):
    return np.exp(-t**2/(2*sigmalo**2))*(t < 0) + np.exp(-t**2/(2*sigmahi**2))*(t >= 0)

def norm(v, vnorm=None):
    if vnorm is None:
        vnorm = v
    if IPLANE == 2:
        return v / vnorm.max()
    else:
        return v / np.abs(vnorm.min())
    
def center(t, v):
    if IPLANE == 0: # Ind-0 -- center on down-peak
        return t - t[np.argmin(v)]
    elif IPLANE == 2: # Collection -- center on peak
        return t - t[np.argmax(v)]
    else: # Ind-1 -- center on zero-cross
        center_ind = np.argmin(np.abs(v[np.argmax(v):np.argmin(v)])) + np.argmax(v)
        return t - t[center_ind]

In [ ]:
#Old electronics response lookup table
ER_val = fERh.values()
ER_time = fERh.axis().centers()
ER = interpolate.interp1d(ER_time, ER_val, kind="linear", bounds_error=False, fill_value=0)

#Parametrized electronics response
er_width = 1.3
def electronics_response(t,width):
    mu = 0
    amp = 10.
    y = (t>=mu)*amp*(1-np.exp(-0.5*(0.9*(t-mu)/width)**2))*np.exp(-0.5*(0.5*(t-mu)/width)**2)
    return y

plt.plot(time, electronics_response(time,er_width) / electronics_response(time,er_width).max())
#plt.plot(ER_time, ER_val / ER_val.max())
plt.xlabel("Time [$\\mu$s]")
plt.ylabel("Amplitude")
plt.xlim([0, 10])
plt.title("Electronics Response")
plt.tight_layout()

if dosave: plt.savefig(savedir + "elecResp.pdf")

In [ ]:
dat = json.load(open(WCpaths_filename_nom))

driftV = dat["FieldResponse"]["speed"]*1e3 # mm/us

thispaths = dat["FieldResponse"]["planes"][IPLANE]["PlaneResponse"]["paths"]

pitchpos_f = [path["PathResponse"]["pitchpos"] for path in thispaths] # mm

paths_f = [np.array(path["PathResponse"]["current"]["array"]["elements"])
             for path in thispaths]

pitchpos = []
paths_nom = []

for i, (ppos, path) in enumerate(zip(pitchpos_f, paths_f)):
    pitchpos.append(ppos)
    paths_nom.append(path)
    if -ppos not in pitchpos_f:
        pitchpos.append(-ppos)
        paths_nom.append(path)
    # handle boundary between (half-)wires
    else:
        paths_nom[-1] = tuple([paths_nom[-1], paths_f[pitchpos_f.index(-ppos)]])
        
pitchpos, paths_nom = zip(*sorted(zip(pitchpos, paths_nom), key=lambda pair: pair[0]))
pitchpos = np.array(pitchpos)

pitchpos_interp = np.linspace(pitchpos.min(), pitchpos.max(), 2101) + 0.015

In [ ]:
dat = json.load(open(WCpaths_filename_noY))

driftV = dat["FieldResponse"]["speed"]*1e3 # mm/us

thispaths = dat["FieldResponse"]["planes"][IPLANE]["PlaneResponse"]["paths"]

pitchpos_f = [path["PathResponse"]["pitchpos"] for path in thispaths] # mm

paths_f = [np.array(path["PathResponse"]["current"]["array"]["elements"])
             for path in thispaths]

pitchpos = []
paths_noY = []

for i, (ppos, path) in enumerate(zip(pitchpos_f, paths_f)):
    pitchpos.append(ppos)
    paths_noY.append(path)
    if -ppos not in pitchpos_f:
        pitchpos.append(-ppos)
        paths_noY.append(path)
    # handle boundary between (half-)wires
    else:
        paths_noY[-1] = tuple([paths_noY[-1], paths_f[pitchpos_f.index(-ppos)]])
        
pitchpos, paths_noY = zip(*sorted(zip(pitchpos, paths_noY), key=lambda pair: pair[0]))
pitchpos = np.array(pitchpos)

pitchpos_interp = np.linspace(pitchpos.min(), pitchpos.max(), 2101) + 0.015

In [ ]:
for p, path in zip(pitchpos, paths_nom):
    if isinstance(path, tuple):
        plt.plot(time, -path[0])
    else:
        plt.plot(time, -path)
    
plt.title("Wire-Cell Electron Path Responses")
plt.xlabel("Time [$\\mu$s]")
plt.ylabel("Current")

plt.xlim([58 - 2*(2-IPLANE), 66 - 2*(2-IPLANE)])
plt.tight_layout()

plt.axvline([time[CENTER_IND]], color="r")

if dosave: plt.savefig(savedir + "pathResp.pdf")

In [ ]:
for p, path in zip(pitchpos, paths_noY):
    if isinstance(path, tuple):
        plt.plot(time, -path[0])
    else:
        plt.plot(time, -path)
    
plt.title("Wire-Cell Electron Path Responses")
plt.xlabel("Time [$\\mu$s]")
plt.ylabel("Current")

plt.xlim([58 - 2*(2-IPLANE), 66 - 2*(2-IPLANE)])
plt.tight_layout()

plt.axvline([time[CENTER_IND]], color="r")

if dosave: plt.savefig(savedir + "pathResp_noY.pdf")

In [ ]:
if IPLANE == 2:
    txt_x = 0.025
else:
    txt_x = 0.55

In [ ]:
# Pick plane, load in data, etc

# Filename with measured signal response
file_path = "../data/run_"
file_path = "./data/far_TrackBased_official/"

data_files = []
run_list = ["8749","9133"]
tpc_list = ["EE","EW","WE","WW"]
for run in run_list:
    for tpc in tpc_list:
        data_files.append(uproot.open(file_path+run+"/WFresults_Plane"+str(IPLANE)+"_"+tpc+".root"))
        
#Create arrays over angle bins to store angle info, data filenames, data products
angle_min = 20
angle_max = 80
angle_increment = 2

nall_angles = len(all_angles)
nfiles = len(data_files)
ndata = 401
data_hists_allfiles = [[[0.0 for k in range(ndata)] for j in range(nall_angles)] for i in range(nfiles)]
data_hists    = [[0.0 for k in range(ndata)] for j in range(nall_angles)]
data_err_vecs = [[0.0 for k in range(ndata)] for j in range(nall_angles)]

data_times = []
data_when_list = []
data_fit_list = []

for j,angle in enumerate(all_angles):
    thlo = angle
    thhi = thlo + angle_increment
    
    angle_data_hists = []
    angle_err_hists = []
    for i, uhf in enumerate(data_files):
        data = uhf["AnodeRecoHist1D_%ito%i" % (thlo, thhi)]
        data_err = uhf["AnodeTrackUncertHist2D_%ito%i" % (thlo, thhi)].to_numpy()[0][5]
        angle_data_hists.append(data.values())
        angle_err_hists.append(data_err)
        data_hists[j] += data.values()
        data_err_vecs[j] = np.sqrt(np.square(data_err_vecs[j])+np.square(data_err))
        if i == 0:
            data_times.append(center(data.axis().centers(), data.values()))
            data_when_list.append(np.abs(data_times[-1]) < PLT_PERIOD)
            data_fit_list.append(np.abs(data_times[-1]) < FIT_PERIOD)

    # NOTE: use std-dev to get uncertainty -- ignore intrinsic error
    data_hists[j][:] = np.average(angle_data_hists, axis=0)
    data_err_vecs[j][:] = np.std(angle_data_hists, axis=0)

In [ ]:
#Helper function to compute chi2
def calc_chi2(pred, meas, err):
    return np.sum(((meas - pred)/err)**2)

#Convert from prepared time (containing angle info) to actual time
def convert_time(t0):
    angle_bin = np.floor(t0/(2*PLT_PERIOD)).astype(int)
    angle_bin_set = to_set(angle_bin)
    angle_bin_index = np.zeros(len(angle_bin),dtype=int)
    for i in range(len(angle_bin_set)):
        angle_bin_index += i*(angle_bin==angle_bin_set[i]).astype(int)
    t = (t0 % (2*PLT_PERIOD)) - PLT_PERIOD
    angle = (all_angles[angle_bin_set]+all_angles[angle_bin_set+1])/2
    
    return (t,angle_bin_set,angle_bin_index,angle)

def contains(array,x):
    return np.any(array == x)

#returns the set of elements in the input array - no duplicates
def to_set(array):
    return np.unique(array)

#Takes as input the scaled times for each path, the WC paths and the interpolated path positions and returns a list of interpolated paths
def interpolate_paths(paths, pitchpos_interp):
    paths_interp = []

    for j, p in enumerate(pitchpos_interp):
        i_pitchpos = int((p - pitchpos[0]+1e-6) / (pitchpos[1] - pitchpos[0]))
        if i_pitchpos == len(pitchpos) - 1:
            path = paths[i_pitchpos]
        else:
            F1 = paths[i_pitchpos]
            F2 = paths[i_pitchpos+1]

            # handle boundary between (half-)wires
            if isinstance(F1, tuple):
                if p > pitchpos[i_pitchpos]:
                    F1 = F1[0]
                else:
                    F1 = F1[1]
            if isinstance(F2, tuple):
                if p > pitchpos[i_pitchpos+1]:
                    F2 = F2[0]
                else:
                    F2 = F2[1]

            interp = (pitchpos[i_pitchpos+1] - p)/(pitchpos[1] - pitchpos[0])
            path = F1*interp + F2*(1-interp)

        paths_interp.append(path)
    return paths_interp

def shifted_paths_sum(angle, paths_interp, pitchpos_interp):
    thxw = angle*np.pi/180
    shift = (np.outer(np.tan(thxw),pitchpos_interp/driftV/dt)).astype(int)
    summed_paths_angle = []
    for i in range(len(angle)):
        summed_paths_angle.append(np.zeros(paths_interp[0].shape))
        
        for j in range(len(paths_interp)):
            s = shift[i][j]
            if s < 0:
                summed_paths_angle[-1][:s] += -paths_interp[j][-s:]
            elif s > 0:
                summed_paths_angle[-1][s:] += -paths_interp[j][:-s]
            else:
                summed_paths_angle[-1] += -paths_interp[j]
                
    return summed_paths_angle

#Finds the index where a strictly increasing array first goes above 0
def find_zero(array):
    return np.argmax(array >= 0)

#Scale the array of input times t by values sl and sr for the left and right sides of 0, respectively
def scale_time(t,sl,sr):
    t0_index = find_zero(t)
    scaled_time = np.zeros(len(t))
    scaled_time[:t0_index] = t[:t0_index]*sl
    scaled_time[t0_index:] = t[t0_index:]*sr
    return scaled_time

#Weights each path based on left and right parameters wl, wr
def weight_path(path, path_time, wl, wr):
    center_index = find_zero(path_time)
    path_left  = path*wl
    path_right = path*wr
    return np.concatenate((path_left[:center_index], path_right[center_index:]))

#Computes a scaled path at given offset by evaluating an input path at times path_time
def scale_path(path, path_time, scale_left, scale_right, wl, wr):
    scaled_time = scale_time(path_time, scale_left, scale_right)
    if isinstance(path, tuple):
        weighted_path_left  = weight_path(path[0],path_time,wl,wr)
        weighted_path_right = weight_path(path[1],path_time,wl,wr)
        pathl_interp = interpolate.interp1d(path_time, weighted_path_left, kind="linear", bounds_error=False, fill_value=0)
        pathr_interp = interpolate.interp1d(path_time, weighted_path_right, kind="linear", bounds_error=False, fill_value=0)
        scaled_path = tuple((pathl_interp(scaled_time),pathr_interp(scaled_time)))
    else:
        weighted_path = weight_path(path,path_time,wl,wr)
        path_interp = interpolate.interp1d(path_time, weighted_path, kind="linear", bounds_error=False, fill_value=0)
        scaled_path = path_interp(scaled_time)
    return scaled_path

#Sum the field responses over drift offset to get the overall field response
def compute_field_response(shifted_paths, angles):
    field_response_angles = []
    for i in range(nangles):
        field_response_angles.append(-sum(shifted_paths[i]))
    return field_response_angles

# TODO FIX -- this doesn't really work
def compute_path_center_ind(path):
    if isinstance(path, tuple):
        v = -path[0]
    else:
        v = -path
        
    if IPLANE == 2:
        center_ind = np.argmax(v)
    elif IPLANE == 0:
        center_ind = np.argmin(v)
    else: # Ind-1 -- center on zero-cross
        center_ind = np.argmax(v[585:] <= 0) + 585
    return center_ind

def compute_path_center_time(path, time):
    return time - time[CENTER_IND]

In [ ]:
centers = []
for p in paths_nom[100:110]:
    centers.append(compute_path_center_ind(p))
centers

In [ ]:
#Function that produces the complete toy mc waveform simulation - interpolated field response, signal response, and gaussian convolution
def mc_waveform_fixsigma(t0, er_width, sl0, sr0, sl1, sr1, weight_ratio, nom_frac, *mus):
    return mc_waveform(t0, er_width, sl0, sr0, sl1, sr1, weight_ratio, nom_frac, SIGMA_A, SIGMA_B, *mus)

def mc_waveform(t0, er_width, sl0, sr0, sl1, sr1, weight_ratio, nom_frac, sigma_a, sigma_b, *mus):
    nangle = len(MC_ANGLES)
    if len(mus) == 1:
        mus = mus*nangle
    else: 
        assert(len(mus) == nangle)
        
    t0_angle = np.split(t0, nangle)
    
    #Compuate scaled times and interpolate to get scaled paths, all for each angle
    offset_param = 1-np.exp(-np.absolute(pitchpos)/1.5)  #3 mm between wires
    scaling_left  = sl1*offset_param
    scaling_right = sr1*offset_param
    weights_left  = (1 - weight_ratio)
    weights_right = 1. / weights_left
    
    scaled_paths = []
    for i, (p_nom, p_noY) in enumerate(zip(paths_nom, paths_noY)):
        if isinstance(p_nom, tuple):
            p = (p_nom[0]*nom_frac + p_noY[0]*(1-nom_frac), 
                 p_nom[1]*nom_frac + p_noY[1]*(1-nom_frac))
        else:
            p = p_nom*nom_frac + p_noY*(1-nom_frac)
        
        sp = scale_path(p, compute_path_center_time(p, time),
                        sl0 + scaling_left[i], sr0 + scaling_right[i],
                        weights_left, weights_right)
        scaled_paths.append(sp)
    
    # interpolate the paths with a finer spacing (0.03mm)
    # avoid edge effects by spacing in between the discontinuities in the paths (every 1.5mm)
    paths_interp = interpolate_paths(scaled_paths, pitchpos_interp)
        
    # Compute the interpolated field response at each track angle
    field_response_angles = shifted_paths_sum(MC_ANGLES, paths_interp, pitchpos_interp)
        
    #Convolve with electronics response and Gaussian smearing
    gaus_sigma = np.sqrt(sigma_a**2 + (np.tan(MC_ANGLES*np.pi/180)*sigma_b)**2)
    
    mc_val_interp_list = []
    vals = []
    for i in range(nangle):
        SR = convolve(field_response_angles[i], electronics_response(time, er_width))
        if gaus_sigma[i] > 1e-4:
            SR_gaus = norm(convolve(SR, gaus(time, gaus_sigma[i])))
        else:
            SR_gaus = SR
        centered_time = center(time, SR_gaus)
        SR_interp = interpolate.interp1d(centered_time + mus[i], SR_gaus, kind="linear", bounds_error=False, fill_value=0)
        vals.append(SR_interp(t0_angle[i]))

    return np.concatenate(vals)

In [ ]:
#Create arrays for time,data in fit and plot regions
time_fitregion_list = []
time_plotregion_list = []

data_fitregion_list = []
data_plotregion_list = []
data_fitregion_norm_list = []
data_plotregion_norm_list = []

data_err_fitregion_list = []
data_err_plotregion_list = []
data_err_fitregion_norm_list = []
data_err_plotregion_norm_list = []

#Loop over angles
nangles = len(FIT_ANGLES)
for i_angle in range(nangles):
    angle = FIT_ANGLES[i_angle]
    i_angle_h = np.where(all_angles == angle)[0][0]
    thlo = angle
    thhi = thlo + angle_increment
    
    time_fitregion_list.append(data_times[i_angle_h][data_fit_list[i_angle_h]])
    time_plotregion_list.append(data_times[i_angle_h][data_when_list[i_angle_h]])

    data_fitregion_list.append(data_hists[i_angle_h][data_fit_list[i_angle_h]])
    data_plotregion_list.append(data_hists[i_angle_h][data_when_list[i_angle_h]])
    data_fitregion_norm_list.append(norm(data_fitregion_list[-1]))
    data_plotregion_norm_list.append(norm(data_plotregion_list[-1]))

    data_err_fitregion_list.append(data_err_vecs[i_angle_h][data_fit_list[i_angle_h]])
    data_err_plotregion_list.append(data_err_vecs[i_angle_h][data_when_list[i_angle_h]])
    data_err_fitregion_norm_list.append(norm(data_err_fitregion_list[-1], data_fitregion_list[-1]))
    data_err_plotregion_norm_list.append(norm(data_err_plotregion_list[-1], data_plotregion_list[-1]))

time_fitregion           = np.concatenate(time_fitregion_list)
time_plotregion          = np.concatenate(time_plotregion_list)

data_fitregion           = np.concatenate(data_fitregion_list)
data_plotregion          = np.concatenate(data_plotregion_list)
data_fitregion_norm      = np.concatenate(data_fitregion_norm_list)
data_plotregion_norm     = np.concatenate(data_plotregion_norm_list)

data_err_fitregion       = np.concatenate(data_err_fitregion_list)
data_err_plotregion      = np.concatenate(data_err_plotregion_list)
data_err_fitregion_norm  = np.concatenate(data_err_fitregion_norm_list)
data_err_plotregion_norm = np.concatenate(data_err_plotregion_norm_list)

#Fit
do_fit = True
nparams = 7
p0 = [0]*(nparams)
p0[0] = 1.3     #Default electronics response width of 1.3
p0[1] = 1     #Scale time left
p0[2] = 1     #Scale time right
p0[3] = 0     #Scale time left offset dependence
p0[4] = 0     #Scale time right offset dependence
p0[5] = 0      #Scale waveform left v right
p0[6] = 0.9      # Fraction of nominal v no-Y config

nmu = nangles
# nmu = 1

mu0 = [0]*nmu
mu_lo = [-2]*nmu
mu_hi = [2]*nmu

p0 += mu0

bound_lo = [1.2999, 0.2, 0.2, -0.2, -0.2, -0.8, 0]
bound_lo += mu_lo
bound_hi = [1.3001, 5.0, 5.0,  5.0,  5.0,  0.8, 1]
bound_hi += mu_hi

parameter_names = [
    "tau",
    "dilate left",
    "dilate right",
    "dilate left pitch",
    "dilate right pitch",
    "scale left-right",
    "nom frac"
] + ["mu%i" % i for i in range(nmu)]

MC_ANGLES = FIT_ANGLES

MC_ANGLES

In [ ]:
debugging = True

#Default prediction for comparison
do_fit = False
pred_default_plotregion = mc_waveform_fixsigma(time_plotregion, *p0)
pred_default_plotregion_norm = norm(pred_default_plotregion)

pred_default_plotregion_norm_list = np.split(pred_default_plotregion_norm, nangles)

pred_default_plotregion_weird = mc_waveform_fixsigma(time_plotregion, 1.3, 1, 1, 0, 0.8, 0, 0.8, 0)
pred_default_plotregion_weird_norm = norm(pred_default_plotregion_weird)

pred_default_plotregion_norm_weird_list = np.split(pred_default_plotregion_weird_norm, nangles)

for i,a in enumerate(FIT_ANGLES):
    thlo = a
    thhi = thlo + angle_increment
    
    plt.figure(i)
    plt.plot(time_plotregion_list[i], pred_default_plotregion_norm_list[i])
    plt.plot(time_plotregion_list[i], pred_default_plotregion_norm_weird_list[i])
    plt.text(txt_x, 0.8, "$%i^\\circ < \\theta_{xw} < %i^\\circ$" % (thlo, thhi),
             fontsize=16,transform=plt.gca().transAxes)
    
    plt.errorbar(time_plotregion_list[i], data_plotregion_norm_list[i], data_err_plotregion_norm_list[i],
             linestyle="none", marker=".", color="black", label="Data")

In [ ]:
%timeit mc_waveform_fixsigma(time_fitregion, *p0)

In [ ]:
cProfile.run('mc_waveform_fixsigma(time_fitregion, *p0)', sort="time")

In [ ]:
debugging = False

print('fitting')
popt, perr = curve_fit(mc_waveform_fixsigma, time_fitregion, data_fitregion_norm, 
                       p0=p0, sigma=data_err_fitregion_norm, absolute_sigma=True, bounds=(bound_lo, bound_hi))

for ip in range(len(p0)):
    print(parameter_names[ip], p0[ip], bound_lo[ip], bound_hi[ip], popt[ip])

#Create arrays for mc in fit and plot regions
print('pred')
pred_fitregion  = mc_waveform_fixsigma(time_fitregion, *popt)
pred_plotregion = mc_waveform_fixsigma(time_plotregion, *popt)
pred_fitregion_norm = norm(pred_fitregion)
pred_plotregion_norm = norm(pred_plotregion)

pred_fitregion_norm_list = np.split(pred_fitregion_norm, nangles)
pred_plotregion_norm_list = np.split(pred_plotregion_norm, nangles)

chi2 = calc_chi2(pred_fitregion_norm, data_fitregion_norm, data_err_fitregion_norm)
ndf = len(pred_fitregion_norm)
print('fit chi2/ndf =     '+str(chi2)+' / '+str(ndf))

In [ ]:
debugging = True

#Default prediction for comparison
do_fit = False
pred_default_fitregion  = mc_waveform_fixsigma(time_fitregion, *(list(p0[:nparams]) + list(popt[nparams:])))
pred_default_plotregion = mc_waveform_fixsigma(time_plotregion, *(list(p0[:nparams]) + list(popt[nparams:])))
pred_default_fitregion_norm = norm(pred_default_fitregion)
pred_default_plotregion_norm = norm(pred_default_plotregion)

pred_default_fitregion_norm_list = np.split(pred_default_fitregion_norm, nangles)
pred_default_plotregion_norm_list = np.split(pred_default_plotregion_norm, nangles)

chi2_default = calc_chi2(pred_default_fitregion_norm,data_fitregion_norm,data_err_fitregion_norm)
print('default chi2/ndf = '+str(chi2_default)+' / '+str(ndf))

#Ratio of Default/Fit MC
pred_ratio_plotregion_norm = pred_default_plotregion_norm/pred_plotregion_norm
pred_ratio_plotregion_norm_list = np.split(pred_ratio_plotregion_norm, nangles)

In [ ]:
#Draw subplots
for i,a in enumerate(FIT_ANGLES):
    thlo = a
    thhi = thlo + angle_increment
    plt.figure(i+1)
    
    fig,(ax1,ax2) = plt.subplots(num=i+1,figsize=(6.4, 5.6),nrows=2, sharex=True, gridspec_kw={'height_ratios': [3, 1]})

    ax1.errorbar(time_plotregion_list[i], data_plotregion_norm_list[i], data_err_plotregion_norm_list[i],
                 linestyle="none", marker=".", color="black", label="Data")
    ax1.plot(time_plotregion_list[i], pred_default_plotregion_norm_list[i], 
             label="Toy Response Default")
    ax1.plot(time_plotregion_list[i], pred_plotregion_norm_list[i], 
             label="Toy Response w/ Fit",color='orange')
    ax1.legend(ncol=2 ,loc='upper center', bbox_to_anchor=(0.5, 1.35))
    
    ax2.errorbar(time_plotregion_list[i], data_plotregion_norm_list[i]/pred_plotregion_norm_list[i],
             abs(data_err_plotregion_norm_list[i]/pred_plotregion_norm_list[i]),linestyle="none", marker=".", color="black")
    ax2.set_ylim([0.8, 1.2])
    ax2.axhline(1, color='orange')
    ax2.plot(time_plotregion_list[i], pred_ratio_plotregion_norm_list[i], label="Default/Fit MC Ratio")
    
    plt.xlim([-5, 5])
    plt.subplots_adjust(hspace=0)
    ax1.text(txt_x, 0.85, "$%i^\\circ < \\theta_{xw} < %i^\\circ$" % (thlo, thhi),fontsize=16,transform=ax1.transAxes)
    
    if IPLANE == 2: 
        txt_xy = (0.675, 0.6)
    else:
        txt_xy = (0.05, 0.05)
    ax1.text(*txt_xy, "$\\chi^2_{nom}/n:$\n\t$%.0f /(%i - %i)$\n$\\chi^2_{fit}/n:$\n\t$%.0f /(%i - %i)$" % (chi2_default, ndf, nmu, chi2, ndf, len(p0)),
            fontsize=10, transform=ax1.transAxes)
    
    ax2.set_xlabel("Time [$\\mu$s]")
    ax1.set_ylabel("Amp. Normalized")
    ax2.set_ylabel("Data / Fit", fontsize=12)
    
    if dosave:
        plt.savefig(savedir + "signalfit_th%ito%i.pdf" % (thlo, thhi))


In [ ]:
# try out plotting some angles to see how the fit compares to the nominal
MC_ANGLES = np.array(list(range(0, 90, 5)))
plt_time = np.linspace(-20, 20, 400)
plt_times = np.tile(plt_time, len(MC_ANGLES))

nom_wvf = mc_waveform(plt_times, *(list([1.3] + p0[1:nparams]) + [0, 0, 0]))
nom_wvfs = np.split(nom_wvf, len(MC_ANGLES))

fit_wvf = mc_waveform(plt_times, *(list(popt[:nparams]) + [0, 0, 0]))
fit_wvfs = np.split(fit_wvf, len(MC_ANGLES))

for i, th in enumerate(MC_ANGLES):
    plt.figure(i)
    plt.plot(plt_time, norm(nom_wvfs[i]), label="Nominal")
    plt.plot(plt_time, norm(fit_wvfs[i]), label="Fit")
    plt.title("Plane %i, $\\theta_{xw} = %i^\\circ$" % (IPLANE, th))
    plt.legend()
    if th < 75:
        plt.xlim([-10, 10])